In [42]:
# Installing and importing all the required libraries
!pip install python-dotenv
import ollama
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
!pip install langchain prompts
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [43]:
# Loading the OpenAI api which is stored as a Colab Secret Key
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
else:
    print("API Key found")

API Key found


In [44]:
# Initializing OpenAI client
openai = OpenAI()

In [45]:
# Creating a class to represent a website, using Beautiful Soup to extract content
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        # Removing irrelevant tags like <script>, <style>, <img>, and <input> before extracting the text.
        for irrelevant in soup.body(["script", "style", "img", "input"]):
          irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [46]:
# Defining the system prompt that guides the chatbot behavior
system_prompt = "You are a chatbot, expert in all the fields that analyzes the contents of a website and provides a short summary making it simple, ignoring text that might be navigation related, focusing mainly on the topic in hand."
system_prompt+="Respond in markdown."

# Generate the user prompt by passing the content of the website
def generate_user_prompt(website):
    user_prompt = f"Refer to the content provided below and write a summary"
    user_prompt += website.text
    return user_prompt

In [47]:
# Generate the complete message structure needed for OpenAI's chat API
def generate_message_prompt(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": generate_user_prompt(website)}
    ]

In [58]:
# Function to summarize the website by calling OpenAI's API
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = generate_message_prompt(website)
    )
    return response.choices[0].message.content

# Display the summary of the website using Markdown for better formatting
def display_summary(url):
  summary = summarize(url)
  display(Markdown(summary))
  return (summary)

In [59]:
openai_result = display_summary("https://en.wikipedia.org/wiki/Generative_artificial_intelligence")

# Summary of Generative Artificial Intelligence

Generative artificial intelligence (GenAI) is a subset of AI that produces new content—such as text, images, and videos—by learning from existing data. The recent advancements in this field, particularly in deep learning and transformer-based models, have led to significant breakthroughs and a surge in applications across various industries like entertainment, healthcare, and finance.

## Historical Context
- **Early Development**: The concept traces back to ancient auto-mata and gained traction in the academic realm during the Dartmouth College workshop in 1956. The rise of generative modeling began substantially in 2014 with the introduction of techniques such as generative adversarial networks (GANs).
- **Recent Boom**: The AI boom starting in 2020 saw the widespread adoption of systems like OpenAI's ChatGPT and image generation models such as DALL-E.

## Modalities and Applications
Generative AI operates across several modalities:
- **Text**: Large language models (LLMs) like GPT-3 and GPT-4 are utilized for natural language generation and processing.
- **Images**: Tools like Midjourney and Stable Diffusion create visual arts based on textual prompts.
- **Audio**: Innovations in voice synthesis and AI-generated music have emerged through platforms like ElevenLabs.
- **Video**: Models like Runway's Gen-2 are capable of generating video content.

## Legal and Ethical Concerns
Various legal challenges exist regarding copyright laws related to the use of copyrighted materials for training generative models, as well as the implications of AI-generated content not qualifying for copyright due to lack of human authorship.

There are also ethical concerns tied to job displacement in sectors like journalism and the biases present in generative models that reflect societal inequalities.

## Societal Impacts
The proliferation of generative AI has incited debates over misinformation, especially concerning deepfakes and fake news, prompting calls for regulation and transparency in AI practices. Various governments and organizations are actively engaging in policy discussions to mitigate these effects.

Overall, while generative AI holds immense potential for creativity and productivity, it also poses significant challenges that society must navigate collaboratively.

In [62]:
OLLAMA_API = "http://localhost:11434/api/chat"
def summarize2(url):
    website = Website(url)
    response=ollama.chat(model="llama3.2", messages=generate_message_prompt(website))
    return response['message']['content']
# Display the summary of the website using Markdown for better formatting
def display_summary_ollama(url):
  summary = summarize2(url)
  display(Markdown(summary))
  return summary

In [63]:
llama_result = display_summary_ollama("https://en.wikipedia.org/wiki/Generative_artificial_intelligence")

The provided text is an article about Generative Artificial Intelligence, which refers to a field of machine learning that involves generating new content, such as images, music, or text, using algorithms and neural networks. Here are some key points from the article:

**Definition**: Generative artificial intelligence (GAI) refers to the development of AI systems that can generate new content, such as images, music, or text, based on patterns and structures learned from existing data.

**Applications**: GAI has a wide range of applications in fields such as art, design, marketing, and entertainment. Examples include:

* Image generation: generating new images using techniques like Generative Adversarial Networks (GANs) and Variational Autoencoders (VAEs)
* Music generation: creating new music using algorithms and neural networks
* Text generation: generating new text, such as articles or social media posts, using language models

**Models**: Some notable models in GAI include:

* Generative Pre-trained Transformer (GPT)
* Large Language Model (LLM)
* Deep learning models like Autoencoders and Convolutional Neural Networks (CNNs)

**Notable Artists**: The article mentions several notable artists who have used GAI to create innovative works, including:

* Refik Anadol
* Cory Arcangel
* Sougwen Chung
* Harold Cohen

**Notable Artworks**: Some notable artworks that use GAI include:

* Edmond de Belamy (a portrait generated by an AI algorithm)
* Jesus Dress Up (a digital artwork created using a Generative Adversarial Network)

**Organizations and Conferences**: The article mentions several organizations and conferences related to GAI, including:

* Artfutura
* Artmedia
* Los Angeles Center for Digital Art
* SIGGRAPH

Overall, the article provides an overview of the field of generative artificial intelligence, its applications, models, notable artists, artworks, and organizations.

In [66]:
messages = [
    ("system", "Being the best teacher, compare the two pieces of summaries given and check which one is better. Also give the justification in 5 points, analyzing from the markdown"),
    ("human", "Here's the first summary markdown {openai_result} and here's the second summary markdown {llama_result}"),
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"openai_result": openai_result, "llama_result": llama_result})

In [65]:
model = ChatOpenAI(model = 'gpt-4o-mini')
result = model.invoke(prompt)
print(result.content)

Upon reviewing the two summaries provided, the first summary is better when evaluated on various important aspects. Here are five justification points:

# 1. **Depth of Content**
   - The first summary offers a comprehensive and in-depth exploration of generative artificial intelligence (GenAI). It includes a historical context that traces its development from early concepts to recent advancements, thus providing a more substantial narrative background. In contrast, the second summary presents general information without giving historical insight, which limits its depth.

# 2. **Organization and Structure**
   - The first summary is well-structured with clearly delineated sections (Historical Context, Modalities and Applications, Legal and Ethical Concerns, Societal Impacts), allowing readers to easily navigate through the information. The second summary lacks this organization, presenting bullet points without categorization, making it less reader-friendly and more difficult to compre